<a href="https://colab.research.google.com/github/bhavanabalraj/Federated-Learning/blob/master/Differential%20Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [10]:
import torch

num_entries = 5000

db = torch.rand(num_entries) > 0.5
db.shape

db[0:5]

tensor([False,  True,  True, False, False])

In [12]:
dbList = []

for i in range(num_entries):
  y = db.clone().detach()
  y = torch.cat([y[0:i],y[i+1:]])
  dbList.append(y)

dbList[0].shape

torch.Size([4999])

In [0]:
def get_parallel_db(db, index):
  return torch.cat([db[0:index],db[index+1:]])

def get_parallel_dbs(db):

  parallel_dbs = list()

  for i in range(len(db)):
    pdb = get_parallel_db(db,i)
    parallel_dbs.append(pdb)

  return parallel_dbs

def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  return db, get_parallel_dbs(db)


In [0]:
db, pdbs = create_db_and_parallels(20)
pdbs

In [31]:
def query(db, threshold):
  return (db.sum() > threshold).float()

def sensitivity(query, num_entries):
  
  db, pdbs = create_db_and_parallels(num_entries)

  full_db_result = query(db, 5)

  sensitivity = 0

  for pdb in pdbs:
    pdb_result = query(pdb, 5)
    distance = torch.abs(full_db_result - pdb_result)

    if distance > sensitivity:
      sensitivity = distance

  return sensitivity

sensitivityList = list()

for i in range(10):
  s = sensitivity(query, 10)
  sensitivityList.append(s)

sensitivityList

[0, 0, tensor(1.), 0, 0, tensor(1.), tensor(1.), 0, 0, 0]

In [34]:
def query(db):
  return db.sum()

def differencing(db, pdb):
  full_db_result = query(db)
  pdb_result = query(pdb)
  return full_db_result == pdb_result

db, pdbs = create_db_and_parallels(100)
pdb = get_parallel_db(db, index)
differencing(db, 5)



tensor(False)